In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from faker import Faker

In [2]:
fake = Faker()

In [3]:
TOTAL_ROWS = 500000
PURCHASE_RATIO = 0.85  
ERROR_RATIO = 0.05     
VIEW_RATIO = 0.10 

In [4]:
assert (PURCHASE_RATIO + ERROR_RATIO + VIEW_RATIO) == 1.0

In [5]:
uk_region_countries = ['England', 'Ireland', 'Wales', 'Netherlands', 'Scotland', 'Germany', 'Belgium', 'France']

In [6]:
products = {
    'AI Assistant Pro': {'unit_price': 500.00, 'cost': 200.00, 'category': 'Software', 'performance': 0.5},  
    'AI Assistant Enterprise': {'unit_price': 1300.00, 'cost': 400.00, 'category': 'Software', 'performance': 1.8},  
    'Rapid Prototyping Basic': {'unit_price': 800.00, 'cost': 350.00, 'category': 'Service', 'performance': 0.3},  
    'Rapid Prototyping Advanced': {'unit_price': 2000.00, 'cost': 800.00, 'category': 'Service', 'performance': 2.5}, 
    'DEX Analytics': {'unit_price': 600.00, 'cost': 250.00, 'category': 'Software', 'performance': 1.0},  
    'Employee Experience Suite': {'unit_price': 2500.00, 'cost': 1000.00, 'category': 'Bundle', 'performance': 1.2}  
}

In [7]:
sales_team = {
    'James Wilson': {'performance': 0.7},
    'Sarah Thompson': {'performance': 2.0},
    'Michael OConnor': {'performance': 1.6},
    'Emily Chen': {'performance': 1.4},
    'David MacLeod': {'performance': 1.0},
    'Priya Patel': {'performance': 0.7},
    'Robert Gallagher': {'performance': 0.3},
    'Emma Walsh': {'performance': 2.4}
}

In [8]:
sales_channels = {
    'Website': {'weight': 0.3, 'conversion': 0.15},
    'Email': {'weight': 0.2, 'conversion': 0.25},
    'Social Media': {'weight': 0.1, 'conversion': 0.10},
    'Referral': {'weight': 0.25, 'conversion': 0.30},
    'Direct': {'weight': 0.15, 'conversion': 0.20}
}

In [9]:
error_messages = {
    400: "Bad Request - Invalid Input",
    403: "Forbidden - Access Denied",
    404: "Page Not Found",
    500: "Internal Server Error",
    302: "Temporary Redirect",
    503: "Service Unavailable"
}

In [10]:
log_data = []
start_date = datetime(2018, 1, 1)
end_date = datetime(2023, 12, 31)
date_range = (end_date - start_date).days

In [11]:
for _ in range(TOTAL_ROWS):
    # Generate random timestamp between 2018-2023
    random_day = random.randint(0, date_range)
    random_second = random.randint(0, 86400)
    timestamp = (start_date + timedelta(days=random_day, seconds=random_second)).strftime('%Y-%m-%d %H:%M:%S')
    
    # Determine log type based on ratios
    log_type = random.choices(
        ['PURCHASE', 'ERROR', 'VIEW'],
        weights=[PURCHASE_RATIO, ERROR_RATIO, VIEW_RATIO],
        k=1
    )[0]
    
    # Select country weighted towards UK and Ireland
    country = random.choices(
        uk_region_countries,
        weights=[0.05, 0.05, 0.10, 0.20, 0.10, 0.15, 0.25, 0.10],
        k=1
    )[0]
    
    ip = fake.ipv4()
    method = random.choice(['GET', 'POST'])
    
    if log_type == 'PURCHASE':
        product_name = random.choice(list(products.keys()))
        product_data = products[product_name]
        
        # Determine quantity with higher probability of purchases (at least 1)
        quantity_options = [1, 2, 3, 4, 5]
        quantity_weights = [0.4, 0.3, 0.15, 0.1, 0.05]
        quantity = random.choices(quantity_options, weights=quantity_weights, k=1)[0]
        
        unit_price = product_data['unit_price']
        total = round(unit_price * quantity, 2)
        production_cost = round(product_data['cost'] * quantity, 2)
        
        # Calculate profit strictly as Total Price - Production Cost
        profit = round(total - production_cost, 2)
        
        # Randomly assign sales rep (performance does not affect profit)
        sales_rep = random.choice(list(sales_team.keys()))
        
        # Select sales channel weighted by their weights
        channel = random.choices(
            list(sales_channels.keys()),
            weights=[c['weight'] for c in sales_channels.values()],
            k=1
        )[0]
        
        status = 200
        action = 'PURCHASE'
        endpoint = f"/purchase/confirm/{product_name.lower().replace(' ', '-')}"
    
    elif log_type == 'ERROR':
        error_code = random.choice(list(error_messages.keys()))
        status = error_code
        product_name = ''
        quantity = 0
        unit_price = 0.00
        total = 0.00
        production_cost = 0.00
        profit = 0.00
        sales_rep = ''
        channel = ''
        action = error_messages[error_code]
        endpoint = random.choice([
            '/index.html', '/products', '/checkout',
            '/contact', '/about', '/admin'
        ])
    
    else:  # VIEW
        status = random.choices([200, 304], weights=[0.9, 0.1], k=1)[0]
        product_name = ''
        quantity = 0
        unit_price = 0.00
        total = 0.00
        production_cost = 0.00
        profit = 0.00
        sales_rep = ''
        channel = ''
        action = 'BROWSE'
        endpoint = random.choice([
            '/index.html', '/about', '/contact',
            '/resources', '/blog', '/events'
        ])
    
    log_data.append([
        timestamp, ip, country, method, endpoint, 
        status, product_name, quantity, unit_price,
        total, production_cost, profit, sales_rep, channel, action
    ])

In [12]:
columns = [
    'Timestamp', 'IP_Address', 'Country', 'Method', 'Endpoint',
    'Status_Code', 'Product', 'Quantity', 'Unit_Price',
    'Total_Price', 'Cost', 'Profit', 'Sales_Rep', 'Sales_Channel', 'Action'
]

In [13]:
df = pd.DataFrame(log_data, columns=columns)

In [14]:
df.loc[df['Status_Code'] != 200, ['Quantity', 'Unit_Price', 'Total_Price', 'Cost', 'Profit']] = 0

In [15]:
df.loc[df['Status_Code'] != 200, ['Quantity', 'Unit_Price', 'Total_Price', 'Cost', 'Profit']] = 0
financial_cols = ['Unit_Price', 'Total_Price', 'Cost', 'Profit']
df[financial_cols] = df[financial_cols].round(2)

In [16]:
df.to_csv("//Users//tshephangchepete//Downloads//PD//sales_dataset.csv", index=False)